# Perceptrons

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.data import Dataset

In [0]:
mnist_dataframe = pd.read_csv(
  "https://download.mlcc.google.com/mledu-datasets/mnist_train_small.csv",
  sep=",",
  header=None)
mnist_dataframe = mnist_dataframe.reindex(np.random.permutation(mnist_dataframe.index))

In [0]:
# parse data
def label_feature(dataframe):
  labels = np.matrix.transpose(np.matrix(dataframe.iloc[:,0]))
  features = np.matrix(dataframe.iloc[:,1:785])
  features = features/255  # feature scaling
  return labels, features

In [0]:
# parse label data

def one_hot(oringinal_label):
  datalength = len(oringinal_label)
  label = np.zeros([datalength,10])
  for i in range(len(oringinal_label)):
    index = oringinal_label[i] 
    label[i,index]=1
    
  return label

In [0]:
t_label, train_feature = label_feature(mnist_dataframe.iloc[0:15000,:])
v_label, validation_feature = label_feature(mnist_dataframe.iloc[15000:20000,:])
train_label = one_hot(t_label)
validation_label = one_hot(v_label)

In [0]:
# hidden layer
def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size],stddev=0.01))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs, Weights, biases

In [0]:
# define accuracy
def accuracy(label, prediction):
  correct = 0
  labidx = np.argmax(label, axis=1)
  prdidx = np.argmax(prediction, axis=1)
  for i in range(len(labidx)):
    if labidx[i]==prdidx[i]:
       correct += 1
      
  return correct/len(labidx)

Define the structure of neural network and the placeholder.

In [0]:
InputNeurons = 784
OutputNeurons = 10
train_keep = 0.65  # dropout parameter, keep how many percentage
learning_rate = 0.01
printstep = 50
traing_epochs = 701

# define placeholder for inputs to network
keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, InputNeurons])
ys = tf.placeholder(tf.float32, [None, OutputNeurons])

# add hidden layer
[l1,W1,b1] = add_layer(xs, InputNeurons, 100, activation_function=tf.nn.relu)
[l2,W2,b2] = add_layer(l1,100,100,activation_function=tf.nn.relu)
[l3,W3,b3] = add_layer(l2,100,100,activation_function=tf.nn.relu)
# add output layer
[prediction,Wp,bp] = add_layer(l3, 100, OutputNeurons, activation_function=tf.nn.softmax)

# the error between prediction and real data
#loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), axis=1))
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=ys,logits=prediction,dim=-1)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

# start of the neural network
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
for i in range(traing_epochs):
    # training
    sess.run(train_step, feed_dict={xs: train_feature, ys: train_label, keep_prob: train_keep})
    if i % printstep == 0:
        # to see the step improvement
        train_loss = sess.run(loss, feed_dict={xs: train_feature, ys: train_label, keep_prob: 1})
        validation_loss = sess.run(loss, feed_dict={xs: validation_feature, ys: validation_label, keep_prob:1})
        print("epoch",i,"validation loss=", validation_loss,"Train Loss=",train_loss)
    
    
train_output = sess.run(prediction, feed_dict={xs: train_feature, ys: train_label, keep_prob: 1})
train_accuracy = accuracy(train_label, train_output)
validation_output = sess.run(prediction, feed_dict={xs: validation_feature, ys: validation_label, keep_prob: 1})
validation_accuracy = accuracy(validation_label, validation_output)
print('train_accuracy=',train_accuracy)
print('validation_accuracy=', validation_accuracy)

epoch 0 validation loss= [2.2965686 2.2965665 2.2973008 ... 2.3062768 2.310087  2.3125663] Train Loss= [2.306294  2.3137279 2.2965646 ... 2.2965746 2.2988396 2.3125706]
epoch 50 validation loss= [2.1509519 2.1362724 2.2917955 ... 2.2342858 2.238496  2.392982 ] Train Loss= [2.4568686 2.2652667 2.1033313 ... 2.1845195 2.2600987 2.2338734]
epoch 100 validation loss= [1.7500379 1.8982992 2.4250584 ... 1.5752904 1.7837152 2.4185586] Train Loss= [2.4611082 2.1052418 1.6980841 ... 1.9705143 1.4710073 1.5835495]
epoch 150 validation loss= [1.5288023 1.8273284 2.3399127 ... 1.4646567 1.4620188 2.4513958] Train Loss= [2.4611266 1.629505  1.5105472 ... 1.6362561 1.4632534 1.4674755]
epoch 200 validation loss= [1.4706869 1.5040737 1.7086388 ... 1.4613745 1.4612246 2.4537241] Train Loss= [2.4611354 1.4663463 1.4673867 ... 1.5496904 1.4614002 1.4612685]
epoch 250 validation loss= [1.4644877 1.4764948 1.5118095 ... 1.461173  1.4612318 2.4597223] Train Loss= [2.4611464 1.461176  1.4635274 ... 1.479248